In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
import time
from scipy import stats
from sklearn.utils.extmath import weighted_mode
from sklearn.neighbors.base import _check_weights, _get_weights, NeighborsBase, KNeighborsMixin, RadiusNeighborsMixin, SupervisedIntegerMixin
from sklearn.base import ClassifierMixin
from sklearn.utils import check_array

In [16]:
class RadiusKNeighborsClassifier(NeighborsBase, RadiusNeighborsMixin,
                                SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, radius=1.0, weights='uniform',
                 algorithm='auto', leaf_size=30, p=2, metric='minkowski',
                 outlier_label=None, metric_params=None, **kwargs):
        self._init_params(radius=radius,
                          algorithm=algorithm,
                          leaf_size=leaf_size,
                          metric=metric, p=p, metric_params=metric_params,
                          **kwargs)
        self.weights = _check_weights(weights)
        self.outlier_label = outlier_label

    def predict_proba(self, X):
        X = check_array(X, accept_sparse='csr')
        n_samples = X.shape[0]

        neigh_dist, neigh_ind = self.radius_neighbors(X)
        print('neigh_dist',neigh_dist)
        print('neigh_ind',neigh_ind)
        print('type',neigh_dist)
        print('type',neigh_ind)
        print('shape',neigh_dist.shape)
        print('shape',neigh_ind,shape)
        neigh_ind = neigh_ind.item()
        neigh_ind = neigh_ind.reshape(1, -1)
        neigh_dist = neigh_dist.item()
        neigh_dist = neigh_dist.reshape(1, -1)
        
        classes_ = self.classes_
        _y = self._y
        if not self.outputs_2d_:
            _y = self._y.reshape((-1, 1))
            classes_ = [self.classes_]

        n_samples = X.shape[0]

        weights = _get_weights(neigh_dist, self.weights)
        if weights is None:
            weights = np.ones_like(neigh_ind)

        all_rows = np.arange(X.shape[0])
        probabilities = []
        for k, classes_k in enumerate(classes_):
            pred_labels = _y[:, k][neigh_ind]
            
            proba_k = np.zeros((n_samples, classes_k.size))
            
            # a simple ':' index doesn't work right
            for i, idx in enumerate(pred_labels.T):  # loop is O(n_neighbors)
                proba_k[all_rows, idx] += weights[:, i]

            # normalize 'votes' into real [0,1] probabilities
            normalizer = proba_k.sum(axis=1)[:, np.newaxis]
            normalizer[normalizer == 0.0] = 1.0
            proba_k /= normalizer

            probabilities.append(proba_k)

        if not self.outputs_2d_:
            probabilities = probabilities[0]

        return probabilities

In [17]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']

In [18]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09] #feature weights
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [19]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [20]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y))]# |
    '''
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    '''
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = RadiusKNeighborsClassifier(radius=Radius, weights='distance', 
                               metric='manhattan')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [21]:
def run_validation():
    # Run validation
    df_train = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20 
    
    print('Preparing train data', flush=True)
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    prediction_result = run_prediction(df_train, df_valiation, n_cell_x, n_cell_y, 5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)

In [22]:
Radius = 0.25
run_validation()

Radius = 0.5
run_validation()

Radius = 0.75
run_validation()

Radius = 1.0
run_validation()

Radius = 1.25
run_validation()

Radius = 1.5
run_validation()

Radius = 1.75
run_validation()

Radius = 2.0
run_validation()

Preparing train data


KeyboardInterrupt: 

Validation result: 0.539376

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)